# Dataset agrupado weather ETO

Librerias

In [1]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)  

import numpy as np
import warnings
warnings.filterwarnings('ignore')

#Data viz
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

#Modelos
import sklearn as sk

#Resultados
import shap

Lectura

In [2]:
df = pd.read_csv('C:/Users/denis/OneDrive/Escritorio/Datathon/datos/DATOS_ETO.txt', sep= '|', header=0 )

Resumen

In [3]:
print(f'Número de filas: {df.shape[0]}, Número de columnas:{df.shape[1]}')
print(f'\nCantidad de duplicados: {df.duplicated().sum()}\n')
print('-------- Tipos de columnas --------')
print(df.dtypes)
print('\n -------- Cantidad de nulos --------')
df.isna().sum()

Número de filas: 51180, Número de columnas:275

Cantidad de duplicados: 0

-------- Tipos de columnas --------
date                            int64
DewpointLocalAfternoonAvg     float64
DewpointLocalAfternoonMax     float64
DewpointLocalAfternoonMin     float64
DewpointLocalDayAvg           float64
                               ...   
WindSpeedLocalNighttimeMin    float64
WindSpeedLocalOvernightAvg    float64
WindSpeedLocalOvernightMax    float64
WindSpeedLocalOvernightMin    float64
ID_ESTACION                     int64
Length: 275, dtype: object

 -------- Cantidad de nulos --------


date                          0
DewpointLocalAfternoonAvg     0
DewpointLocalAfternoonMax     0
DewpointLocalAfternoonMin     0
DewpointLocalDayAvg           0
                             ..
WindSpeedLocalNighttimeMin    0
WindSpeedLocalOvernightAvg    0
WindSpeedLocalOvernightMax    0
WindSpeedLocalOvernightMin    0
ID_ESTACION                   0
Length: 275, dtype: int64

In [4]:
df.head()

,date,DewpointLocalAfternoonAvg,DewpointLocalAfternoonMax,DewpointLocalAfternoonMin,DewpointLocalDayAvg,DewpointLocalDayMax,DewpointLocalDayMin,DewpointLocalDaytimeAvg,DewpointLocalDaytimeMax,DewpointLocalDaytimeMin,DewpointLocalEveningAvg,DewpointLocalEveningMax,DewpointLocalEveningMin,DewpointLocalMorningAvg,DewpointLocalMorningMax,DewpointLocalMorningMin,DewpointLocalNighttimeAvg,DewpointLocalNighttimeMax,DewpointLocalNighttimeMin,DewpointLocalOvernightAvg,DewpointLocalOvernightMax,DewpointLocalOvernightMin,EvapotranspirationLocalAfternoonAvg,EvapotranspirationLocalAfternoonMax,EvapotranspirationLocalAfternoonMin,EvapotranspirationLocalDayAvg,EvapotranspirationLocalDayMax,EvapotranspirationLocalDayMin,EvapotranspirationLocalDaytimeAvg,EvapotranspirationLocalDaytimeMax,EvapotranspirationLocalDaytimeMin,EvapotranspirationLocalEveningAvg,EvapotranspirationLocalEveningMax,EvapotranspirationLocalEveningMin,EvapotranspirationLocalMorningAvg,EvapotranspirationLocalMorningMax,EvapotranspirationLocalMorningMin,EvapotranspirationLocalNighttimeAvg,EvapotranspirationLocalNighttimeMax,EvapotranspirationLocalNighttimeMin,EvapotranspirationLocalOvernightAvg,EvapotranspirationLocalOvernightMax,EvapotranspirationLocalOvernightMin,FeelsLikeLocalAfternoonAvg,FeelsLikeLocalAfternoonMax,FeelsLikeLocalAfternoonMin,FeelsLikeLocalDayAvg,FeelsLikeLocalDayMax,FeelsLikeLocalDayMin,FeelsLikeLocalDaytimeAvg,FeelsLikeLocalDaytimeMax,FeelsLikeLocalDaytimeMin,FeelsLikeLocalEveningAvg,FeelsLikeLocalEveningMax,FeelsLikeLocalEveningMin,FeelsLikeLocalMorningAvg,FeelsLikeLocalMorningMax,FeelsLikeLocalMorningMin,FeelsLikeLocalNighttimeAvg,FeelsLikeLocalNighttimeMax,FeelsLikeLocalNighttimeMin,FeelsLikeLocalOvernightAvg,FeelsLikeLocalOvernightMax,FeelsLikeLocalOvernightMin,GlobalHorizontalIrradianceLocalAfternoonAvg,GlobalHorizontalIrradianceLocalAfternoonMax,GlobalHorizontalIrradianceLocalAfternoonMin,GlobalHorizontalIrradianceLocalDayAvg,GlobalHorizontalIrradianceLocalDayMax,GlobalHorizontalIrradianceLocalDayMin,GlobalHorizontalIrradianceLocalDaytimeAvg,GlobalHorizontalIrradianceLocalDaytimeMax,GlobalHorizontalIrradianceLocalDaytimeMin,GlobalHorizontalIrradianceLocalEveningAvg,GlobalHorizontalIrradianceLocalEveningMax,GlobalHorizontalIrradianceLocalEveningMin,GlobalHorizontalIrradianceLocalMorningAvg,GlobalHorizontalIrradianceLocalMorningMax,GlobalHorizontalIrradianceLocalMorningMin,GlobalHorizontalIrradianceLocalNighttimeAvg,GlobalHorizontalIrradianceLocalNighttimeMax,GlobalHorizontalIrradianceLocalNighttimeMin,GlobalHorizontalIrradianceLocalOvernightAvg,GlobalHorizontalIrradianceLocalOvernightMax,GlobalHorizontalIrradianceLocalOvernightMin,GustLocalAfternoonAvg,GustLocalAfternoonMax,GustLocalAfternoonMin,GustLocalDayAvg,GustLocalDayMax,GustLocalDayMin,GustLocalDaytimeAvg,GustLocalDaytimeMax,GustLocalDaytimeMin,GustLocalEveningAvg,GustLocalEveningMax,GustLocalEveningMin,GustLocalMorningAvg,GustLocalMorningMax,GustLocalMorningMin,GustLocalNighttimeAvg,GustLocalNighttimeMax,GustLocalNighttimeMin,GustLocalOvernightAvg,GustLocalOvernightMax,GustLocalOvernightMin,MSLPLocalAfternoonAvg,MSLPLocalAfternoonMax,MSLPLocalAfternoonMin,MSLPLocalDayAvg,MSLPLocalDayMax,MSLPLocalDayMin,MSLPLocalDaytimeAvg,MSLPLocalDaytimeMax,MSLPLocalDaytimeMin,MSLPLocalEveningAvg,MSLPLocalEveningMax,MSLPLocalEveningMin,MSLPLocalMorningAvg,MSLPLocalMorningMax,MSLPLocalMorningMin,MSLPLocalNighttimeAvg,MSLPLocalNighttimeMax,MSLPLocalNighttimeMin,MSLPLocalOvernightAvg,MSLPLocalOvernightMax,MSLPLocalOvernightMin,PrecipAmountLocalAfternoonAvg,PrecipAmountLocalAfternoonMax,PrecipAmountLocalAfternoonMin,PrecipAmountLocalDayAvg,PrecipAmountLocalDayMax,PrecipAmountLocalDayMin,PrecipAmountLocalDaytimeAvg,PrecipAmountLocalDaytimeMax,PrecipAmountLocalDaytimeMin,PrecipAmountLocalEveningAvg,PrecipAmountLocalEveningMax,PrecipAmountLocalEveningMin,PrecipAmountLocalMorningAvg,PrecipAmountLocalMorningMax,PrecipAmountLocalMorningMin,PrecipAmountLocalNighttimeAvg,PrecipAmountLocalNighttimeMax,PrecipAmountLocalNi

Nulos

In [5]:
nulos = pd.DataFrame(df.isna().sum(),columns=['nulls'])
nulos[nulos.nulls>0].sort_values('nulls',ascending=False)

,nulls
GustLocalOvernightMin,44579
GustLocalOvernightMax,44579
GustLocalOvernightAvg,44579
GustLocalMorningMin,43311
GustLocalMorningMax,43311
...,...
FeelsLikeLocalMorningMin,20
FeelsLikeLocalMorningMax,20
FeelsLikeLocalMorningAvg,20
DewpointLocalMorningMin,20


Fecha

In [6]:
df[['date']] = df[['date']].applymap(str).applymap(lambda x: "{}/{}/{}".format(x[4:6],x[6:], x[0:4]))

In [7]:
df['date'] = pd.to_datetime(df.date)

Año-Mes

In [8]:
df['mes']= df.date.dt.month_name()
df['año']= df.date.dt.year.astype("int32")

## Agrupacion

Hay **13 variables**, cada una con 21 combinaciones posibles.

- Período: Afternoon, Day, Daytime, Evening, Morning, Nighttime, Overnight (7)
- Avg, Min, Max (3)

3*7 = 21

**Para el join**

- Elegir una de las 3 métricas para cada una de las 13 variables.
- Elegir una de los 7 períodos para cada variable (1).
- Resulta en 13 variables, con una columna para cada mes, es decir 13 * 1 * 12 = **156 columnas a agregar**

**Join**

- **On**: Año=campaña, id_estacion=id_estacion
- **Values**:  156 columnas

Hace falta agrupar por año, mes y estacion. En este caso calculamos la media unicamente.

In [9]:
agrupado = df.groupby(['año','mes','ID_ESTACION']).mean()
agrupado.reset_index(inplace=True)

Filtramos las columnas que sean diarias y avg

In [10]:
avg_cols = list(agrupado.filter(like='Avg').columns)
avg_cols_days = list(agrupado.filter(like='DayAvg').columns)
ids = ['año','mes','ID_ESTACION']
agrupado_avg = agrupado[ids + avg_cols_days]

Pivote pivote, llamamos join al dataset final

In [11]:
join = agrupado_avg.pivot(index=['año','ID_ESTACION'], columns='mes', values=avg_cols_days).reset_index()

Arreglamos columnas y visualizamos

In [12]:
join.columns = [x+'_'+y if y != '' else x for x,y in join.columns.to_flat_index()]
join.head()

,año,ID_ESTACION,DewpointLocalDayAvg_April,DewpointLocalDayAvg_August,DewpointLocalDayAvg_December,DewpointLocalDayAvg_February,DewpointLocalDayAvg_January,DewpointLocalDayAvg_July,DewpointLocalDayAvg_June,DewpointLocalDayAvg_March,DewpointLocalDayAvg_May,DewpointLocalDayAvg_November,DewpointLocalDayAvg_October,DewpointLocalDayAvg_September,EvapotranspirationLocalDayAvg_April,EvapotranspirationLocalDayAvg_August,EvapotranspirationLocalDayAvg_December,EvapotranspirationLocalDayAvg_February,EvapotranspirationLocalDayAvg_January,EvapotranspirationLocalDayAvg_July,EvapotranspirationLocalDayAvg_June,EvapotranspirationLocalDayAvg_March,EvapotranspirationLocalDayAvg_May,EvapotranspirationLocalDayAvg_November,EvapotranspirationLocalDayAvg_October,EvapotranspirationLocalDayAvg_September,FeelsLikeLocalDayAvg_April,FeelsLikeLocalDayAvg_August,FeelsLikeLocalDayAvg_December,FeelsLikeLocalDayAvg_February,FeelsLikeLocalDayAvg_January,FeelsLikeLocalDayAvg_July,FeelsLikeLocalDayAvg_June,FeelsLikeLocalDayAvg_March,FeelsLikeLocalDayAvg_May,FeelsLikeLocalDayAvg_November,FeelsLikeLocalDayAvg_October,FeelsLikeLocalDayAvg_September,GlobalHorizontalIrradianceLocalDayAvg_April,GlobalHorizontalIrradianceLocalDayAvg_August,GlobalHorizontalIrradianceLocalDayAvg_December,GlobalHorizontalIrradianceLocalDayAvg_February,GlobalHorizontalIrradianceLocalDayAvg_January,GlobalHorizontalIrradianceLocalDayAvg_July,GlobalHorizontalIrradianceLocalDayAvg_June,GlobalHorizontalIrradianceLocalDayAvg_March,GlobalHorizontalIrradianceLocalDayAvg_May,GlobalHorizontalIrradianceLocalDayAvg_November,GlobalHorizontalIrradianceLocalDayAvg_October,GlobalHorizontalIrradianceLocalDayAvg_September,GustLocalDayAvg_April,GustLocalDayAvg_August,GustLocalDayAvg_December,GustLocalDayAvg_February,GustLocalDayAvg_January,GustLocalDayAvg_July,GustLocalDayAvg_June,GustLocalDayAvg_March,GustLocalDayAvg_May,GustLocalDayAvg_November,GustLocalDayAvg_October,GustLocalDayAvg_September,MSLPLocalDayAvg_April,MSLPLocalDayAvg_August,MSLPLocalDayAvg_December,MSLPLocalDayAvg_February,MSLPLocalDayAvg_January,MSLPLocalDayAvg_July,MSLPLocalDayAvg_June,MSLPLocalDayAvg_March,MSLPLocalDayAvg_May,MSLPLocalDayAvg_November,MSLPLocalDayAvg_October,MSLPLocalDayAvg_September,PrecipAmountLocalDayAvg_April,PrecipAmountLocalDayAvg_August,PrecipAmountLocalDayAvg_December,PrecipAmountLocalDayAvg_February,PrecipAmountLocalDayAvg_January,PrecipAmountLocalDayAvg_July,PrecipAmountLocalDayAvg_June,PrecipAmountLocalDayAvg_March,PrecipAmountLocalDayAvg_May,PrecipAmountLocalDayAvg_November,PrecipAmountLocalDayAvg_October,PrecipAmountLocalDayAvg_September,RelativeHumidityLocalDayAvg_April,RelativeHumidityLocalDayAvg_August,RelativeHumidityLocalDayAvg_December,RelativeHumidityLocalDayAvg_February,RelativeHumidityLocalDayAvg_January,RelativeHumidityLocalDayAvg_July,RelativeHumidityLocalDayAvg_June,RelativeHumidityLocalDayAvg_March,RelativeHumidityLocalDayAvg_May,RelativeHumidityLocalDayAvg_November,RelativeHumidityLocalDayAvg_October,RelativeHumidityLocalDayAvg_September,SnowAmountLocalDayAvg_April,SnowAmountLocalDayAvg_August,SnowAmountLocalDayAvg_December,SnowAmountLocalDayAvg_February,SnowAmountLocalDayAvg_January,SnowAmountLocalDayAvg_July,SnowAmountLocalDayAvg_June,SnowAmountLocalDayAvg_March,SnowAmountLocalDayAvg_May,SnowAmountLocalDayAvg_November,SnowAmountLocalDayAvg_October,SnowAmountLocalDayAvg_September,TemperatureLocalDayAvg_April,TemperatureLocalDayAvg_August,TemperatureLocalDayAvg_December,TemperatureLocalDayAvg_February,TemperatureLocalDayAvg_January,TemperatureLocalDayAvg_July,TemperatureLocalDayAvg_June,TemperatureLocalDayAvg_March,TemperatureLocalDayAvg_May,TemperatureLocalDayAvg_November,TemperatureLocalDayAvg_October,TemperatureLocalDayAvg_September,UVIndexLocalDayAvg_April,UVIndexLocalDayAvg_August,UVIndexLocalDayAvg_December,UVIndexLocalDayAvg_February,UVIndexLocalDayAvg_January,UVIndexLocalDayAvg_July,UVIndexLocalDayAvg_June,UVIndexLocalDayAvg_March,UVIndexLocalDayAvg_May,UVIndexLocalDayAvg_November,UVIndexLo

In [13]:
join.shape

(160, 158)

Así como esta, ya se puede Joinear. Faltaría: 

- Que hacemos con NAs?
- Tomamos promedios y listo?
- Tomamos Day o alguna de las otras Max, Min, etc?
- Para Nieve, Lluvia, podemos poner 1 si ocurre y 0 si no.